# Vehicle Price Predictions using Supervised Learning

This project explores different ways to predict vehicular prices using Supervised Learning on the kaggle Automotive Price Prediction Dataset, found at this URL: https://www.kaggle.com/datasets/metawave/vehicle-price-prediction

Note that for this project, since the dataset was relatively small, I chose to download the .csv file directly to my machine. I've prevented it from uploading to git using .gitignore. If you would like to replicate this code, you will need to download the csv from the abovementioned kaggle link. 

This is my final project for Introduction to Machine Learning: Supervised Learning, taught at CU Boulder. It requires 3 deliverables:

1. this notebook
2. a video presentation (powerpoint slides will be available on Git)
3. a public GitHub repository, found here: https://github.com/Kate-Zilla/Vehicle-Price-Predictions-Using-Supervised-Learning 

First I'll start with import statements.

In [1]:
import scipy as sp
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import statsmodels.formula.api as smf
import statsmodels.api as sm
import os


#EDA Section
Here I load & inspect the data


In [2]:

df = pd.read_csv('vehicle_price_prediction.csv')
df.head()

,make,model,year,mileage,engine_hp,transmission,fuel_type,drivetrain,body_type,exterior_color,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,mileage_per_year,brand_popularity,price
0,Volkswagen,Jetta,2016,183903,173,Manual,Electric,RWD,Sedan,Blue,Brown,5,NaN,Dealer,Excellent,EX,9,20433.666667,0.040054,7208.52
1,Lexus,RX,2010,236643,352,Manual,Gasoline,FWD,Sedan,Silver,Beige,5,Minor,Dealer,Good,LX,15,15776.200000,0.039921,6911.81
2,Subaru,Crosstrek,2016,103199,188,Automatic,Diesel,AWD,Sedan,Silver,Beige,5,NaN,Dealer,Excellent,Touring,9,11466.555556,0.040230,11915.63
3,Cadillac,Lyriq,2016,118889,338,Manual,Gasoline,AWD,SUV,Black,Gray,3,NaN,Private,Good,Base,9,13209.888889,0.039847,25984.79
4,Toyota,Highlander,2018,204170,196,Manual,Diesel,FWD,Sedan,Red,Brown,5,Minor,Dealer,Excellent,Sport,7,29167.142857,0.039627,8151.30


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   make              1000000 non-null  object 
 1   model             1000000 non-null  object 
 2   year              1000000 non-null  int64  
 3   mileage           1000000 non-null  int64  
 4   engine_hp         1000000 non-null  int64  
 5   transmission      1000000 non-null  object 
 6   fuel_type         1000000 non-null  object 
 7   drivetrain        1000000 non-null  object 
 8   body_type         1000000 non-null  object 
 9   exterior_color    1000000 non-null  object 
 10  interior_color    1000000 non-null  object 
 11  owner_count       1000000 non-null  int64  
 12  accident_history  249867 non-null   object 
 13  seller_type       1000000 non-null  object 
 14  condition         1000000 non-null  object 
 15  trim              1000000 non-null  object 
 16  v

so we have 20 columns (including price), that contain either object, int, or float data types. note that there is not an ID column. There are 1000000 entries, 0 to 999999

Below I will display the unique values that exist in each column with dtype = object

In [4]:
for c in df.columns:
    if df[c].dtype == 'object':
        print(c, df[c].unique())

make ['Volkswagen' 'Lexus' 'Subaru' 'Cadillac' 'Toyota' 'Land Rover' 'Mazda'
 'Ram' 'Chrysler' 'GMC' 'Volvo' 'Audi' 'Chevrolet' 'Tesla' 'Hyundai'
 'Ford' 'Porsche' 'Acura' 'Nissan' 'Kia' 'Jeep' 'BMW' 'Dodge'
 'Mercedes-Benz' 'Honda']
model ['Jetta' 'RX' 'Crosstrek' 'Lyriq' 'Highlander' 'Defender' 'Mazda3' 'Atlas'
 '2500' '300' 'Yukon' 'XT5' 'Range Rover' 'S60' 'Camry' 'Q5' 'Silverado'
 'Model 3' '3500' 'Sonata' 'Camaro' 'Explorer' '911' 'MDX' 'Sentra'
 'Mustang' 'Discovery' 'R8' 'Forte' 'Equinox' 'Model Y' '1500'
 'Grand Cherokee' 'Mazda6' 'M3' 'Malibu' 'Pacifica' 'Panamera'
 'Pathfinder' 'V60' 'A6' 'Sorento' 'Cherokee' 'IS' 'Macan' 'Tiguan'
 'Tucson' '3 Series' 'Durango' 'Titan' 'TLX' 'S-Class' 'E-Class' 'RDX'
 'Corolla' 'RAV4' 'F-150' 'X5' 'Outback' 'Cayenne' 'Rogue' 'Accord'
 'Acadia' 'ES' 'Sportage' 'Tacoma' 'CT5' 'Challenger' 'Sierra' 'Wrangler'
 'Altima' 'Model S' 'GLE' 'NX' 'Tahoe' 'Q7' 'Integra' 'Impreza' 'Terrain'
 'Civic' 'XC60' 'X3' 'Golf' 'Escape' 'XC90' 'Santa Fe' 'CX-9' '

consider dropping the "make" column, as we already have the model, which can be mapped back to make, also drop year, as we have vehicle age, and drop mileage/year as we have mileage and vehicle age. 

In [5]:
df=df.drop(['make','year','mileage_per_year'], axis=1)

In [6]:
df.head()

,model,mileage,engine_hp,transmission,fuel_type,drivetrain,body_type,exterior_color,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,brand_popularity,price
0,Jetta,183903,173,Manual,Electric,RWD,Sedan,Blue,Brown,5,NaN,Dealer,Excellent,EX,9,0.040054,7208.52
1,RX,236643,352,Manual,Gasoline,FWD,Sedan,Silver,Beige,5,Minor,Dealer,Good,LX,15,0.039921,6911.81
2,Crosstrek,103199,188,Automatic,Diesel,AWD,Sedan,Silver,Beige,5,NaN,Dealer,Excellent,Touring,9,0.040230,11915.63
3,Lyriq,118889,338,Manual,Gasoline,AWD,SUV,Black,Gray,3,NaN,Private,Good,Base,9,0.039847,25984.79
4,Highlander,204170,196,Manual,Diesel,FWD,Sedan,Red,Brown,5,Minor,Dealer,Excellent,Sport,7,0.039627,8151.30
